In [8]:
import sys
import numpy as np
import matplotlib.pyplot as plt
# get_ipython().magic(u'matplotlib inline')
from pybloom import BloomFilter
import inspect
import matplotlib.font_manager
from sklearn import svm
import os
import glob
import pickle as pkl
import time
# get_ipython().magic(u'matplotlib notebook')
from itertools import product
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import pickle
from sklearn import mixture
import math
import scipy
typedata = "vdisk"
# typedata = sys.argv[1]
if typedata=="vdisk":
    import vdiskHelper as dataHelper
typemodel = "svmsimple"
# typemodel = sys.argv[2]
if typemodel=="epsilon":
    import epsilonHelper as modelHelper
elif typemodel=="svmsimple":
    import svmsklearncoreHelper as modelHelper
elif typemodel=="libsvmsch":
    import svmlibsvmHelper as modelHelper
elif typemodel=="isolationforest":
    import isolationForestHelper as modelHelper
elif typemodel=="np":
    import npHelper as modelHelper
elif typemodel=="dic":
    import dicHelper as modelHelper






# In[12]:


# def persistModel(models, D, epsilon, agg_thr, box_thr):
#     pickle_out = open("svmModels/svmmodelstandardscaled_"+namePrefix+str(D)+"_"+str(epsilon)+"_"+str(agg_thr)+"_"+str(box_thr),"wb")
#     pkl.dump(models, pickle_out)
#     pickle_out.close()




# In[14]:


def persistBloomFilters(bloomfilters, params):
    pickle_out = open("bloomfilters/bloomfilters_"+namePrefix+modelHelper.paramsToString(params),"wb")
    pkl.dump(bloomfilters, pickle_out)
    pickle_out.close()

def loadBloomFilters(params):
    pickle_out = open("bloomfilters/bloomfilters_"+namePrefix+modelHelper.paramsToString(params),"rb")
    bloomfilters = pkl.load(pickle_out)
    pickle_out.close()
    return bloomfilters


# In[15]:


def persistObservations(obs, params):
    pickle_out = open("Observations/observations_"+namePrefix+modelHelper.paramsToString(params),"wb")
    pkl.dump(obs, pickle_out)
    pickle_out.close()


# In[16]:


def persistTimes(obs):
    pickle_out = open("times/times_"+namePrefix,"wb")
    pkl.dump(obs, pickle_out)
    pickle_out.close()

def getScalers():
    scalers = {}
    for i in range(0,len(ssTableData)):
        thistableData = ssTableData[i]
        scaler = preprocessing.StandardScaler().fit(thistableData)
        scalers[i] = scaler
    return scalers






def mypred(X, clf):
    predictions = modelHelper.predict(clf, X)
    # predictions = clf.predict(np.array(X))
    return predictions

def testIndi(keyPredg, sstableID, models):
    if str(sstableID) in models:
        clf, reqdim= models[str(sstableID)]
        # print("----------strange")
        # print(type(keyPredg))
        # print(keyPredg)
        # print(keyPredg.toarray())
        # print(type([keyPredg]))
        if isinstance(keyPredg, scipy.sparse.csr.csr_matrix):
            # Bracket is removed for sparse case
            prediction = mypred(keyPredg, clf)
        else:
            prediction = mypred(np.array([keyPredg]), clf)
        return prediction[0]
    else:
        print(str(sstableID), ' - sstableid not in models')
        return 0
def testIndi2(keyPredg, sstableID, models):
    if str(sstableID) in models:
        clf, reqdim= models[str(sstableID)]
        prediction = mypred(keyPredg, clf)
        return prediction
    else:
        print(str(sstableID), ' - sstableid not in models')
        return 0

def putIntoBloomFilters(models, oriSsTableDataStr, bagOfWordsModel):
    bloomfilters = {}
    errorsTrain = {}
    for i in range(0,len(oriSsTableDataStr)):
        print("bf creating for ",i)
        orithistableData = oriSsTableDataStr[i]
        thisbaggeddata = bagOfWordsModel.process(orithistableData)
        # thistableDataTranspose0 = thistableData.transpose()[0]
        if removeClassifier:
            newCapacity = max(len(orithistableData),1)
            print("newCapacity ",newCapacity)
            f = BloomFilter(capacity=newCapacity, error_rate=errorRate)
            for j in range(0,len(orithistableData)):
                f.add(orithistableData[i])
            bloomfilters[i] = f
        else:
            # print(thistableData.tolist())
            # print(thisbaggeddata)
            # print(type(thisbaggeddata))
            # print("--")
            # print(thisbaggeddata.todense())
            falseNegativeAns = testIndi2(thisbaggeddata, i, models)
            # print(falseNegativeAns.tolist())
            print("--------------------------")
            # print(np.r_['-1',thistableData, np.transpose(np.array([falseNegativeAns]))].tolist())
        #     print(falseNegativeAns)
            numFalseNeg = falseNegativeAns.tolist().count(-1)
        #     newCapacity = int(capacityBloom*1.0*numFalseNeg/len(thistableData))
            newCapacity = numFalseNeg
            bloomCapacity = max(newCapacity/1,1)
            print("newCapacity ",newCapacity)
            f = BloomFilter(capacity=bloomCapacity, error_rate=errorRate)
            for j in range(0,len(orithistableData)):
                if(falseNegativeAns[j]==-1 or removeClassifier):
        #             print("adding ", thistableDataTranspose0[j])
                    f.add(orithistableData[j])
            bloomfilters[i] = f
        errorsTrain[i] = newCapacity
    return (bloomfilters,errorsTrain)

def performTests(models, bloomfilters, oriSsTableDataStr, bagOfWordsModel):
    # create test data
    # For now test data is all data
    # calculate false positives for test data
    truePositive_c=0
    truePositive_bf=0
    truePositive=0
    falsePositive_c=0
    falsePositive_bf=0
    falsePositive=0
    trueNegative=0
    for i in range(0, len(oriSsTableDataStr)):
        print("on sstable number ", i)
        orithistableData = oriSsTableDataStr[i]
        thisbaggeddata = bagOfWordsModel.process(orithistableData)
        # thistableDataTranspose0 = thistableData.transpose()[0]
        for elemind in range(0,len(oriSsTableDataStr),skipFactor):
            elemOri = orithistableData[elemind]
            elemBagged = thisbaggeddata[elemind]
            for j in range(0,len(oriSsTableDataStr)):
                oriThisSsTableDataStrSet = set(oriSsTableDataStr[j])
                if removeClassifier:
                    cAnswer = False
                else:
                    cAnswer = (testIndi(elemBagged,j, models)==1)
                    # cAnswer = (testIndiEpsilon2(scalers[j].transform([elem]),j, models)==1)
    #             print(cAnswer)
                if not cAnswer:
                    bfAnswer = elemOri in bloomfilters[j]
                    finAnswer = bfAnswer
                else:
                    bfAnswer = None
                    finAnswer = cAnswer
                if finAnswer:
                    if(i==j):
                        #True answer true positive
                        #break because you found answer
                        truePositive+=1
                        if bfAnswer is None:
                            truePositive_c+=1
                        else:
                            truePositive_bf+=1
                        break
                    else:
                        #False answer false positive
                        elemOriStr = ":".join(map(str,map(int, elemOri)))
                        if elemOri in oriThisSsTableDataStrSet:
                            truePositive+=1
                            if bfAnswer is None:
                                truePositive_c+=1
                            else:
                                truePositive_bf+=1
                            break
                        else:
                            falsePositive+=1
                            if bfAnswer is None:
                                falsePositive_c+=1
                            else:
                                falsePositive_bf+=1
                else:
                    if(i==j):
                        print(elem)
                        print(elemind)
                        print(elemOri)
                        print(np.array(elem))
                        print(elem.tolist())
                        print(len(elem))
                        print(elem[0])
                        print(elem[1])
                        print(type(elem))
                        print(elem.shape)
                        print(cAnswer,bfAnswer)
                        print(testIndi(scalers[j].transform(np.array([elem.tolist()]))[0],j, models))
                        print(testIndi(scalers[j].transform([elem.tolist()]),j, models))
                        thisanswerscaled = scalers[j].transform([elem.tolist()])
                        wholetablescaled = scalers[j].transform(thistableData)
                        falseNegativeAns = testIndi(wholetablescaled, j, models)
                        print(thisanswerscaled)
                        print(thisanswerscaled.tolist())
                        print(testIndi2(thisanswerscaled,j,models))
                        print("----")
                        print("type of ori data", thistableData.dtype, wholetablescaled.dtype, elem.dtype, thisanswerscaled.dtype)
                        print(falseNegativeAns.tolist())
                        print("thisanswerscaled is ",thisanswerscaled)
                        print("joined them as ",separator.join(map(str,thisanswerscaled)))
                        print(lololo)
                        # print(np.r_['-1',wholetablescaled, np.transpose(np.array([falseNegativeAns]))].tolist())
                        # outfile = open("ssTableErrorEpsilon", "wb")
                        # pkl.dump(thistableData, pickle_out)
                        # pickle_out.close()
                        np.save("ssTableErrorEpsilonnp", thistableData)
                        raise ValueEroor("not possible")
                    else:
                        #True answer true negative
                        trueNegative+=1
    fprate = falsePositive*1.0/(falsePositive+trueNegative)
    print("fpr is ", fprate)
    return (fprate,
            truePositive_c,truePositive_bf,truePositive,
            falsePositive_c,falsePositive_bf,falsePositive,
            trueNegative)

def trainIndi(nplst1, sstableID, params, models):
    # print("passed mean was ",np.mean(nplst1, axis=0))
    # print("passed sigma was ",np.var(nplst1, axis=0))
    clf = modelHelper.initAndFit(params, nplst1)
    reqdim = nplst1[0].shape[0]
#     print("reqdim is ",reqdim)
    models[str(sstableID)] = (clf, reqdim)
    print('training done with sstableid - ', sstableID)
    return models

def trainFn(params, oriSsTableDataStr, bagOfWordsModel):
    # train the model on generated data
    models = {}
    for i in range(0,len(oriSsTableDataStr)):
        starttime = time.time()
        # thistableData = ssTableData[i]
        baggedData = bagOfWordsModel.process(oriSsTableDataStr[i])
        models = trainIndi(baggedData,i, params, models)
        endtime = time.time()
        print(modelHelper.paramsToString(params),"time taken ", (endtime-starttime))
    return models

createnew = True

def ensemble(params, oriSsTableDataStr, bagOfWordsModel):
    # print(nu,gamma)
    starttime = time.time()
    if createnew:
        models = trainFn(params, oriSsTableDataStr, bagOfWordsModel)
        modelHelper.persistModel(namePrefix,models,params)
        print(params, "inserting into bloom filters")
        (bloomfilters, errorTrain) = putIntoBloomFilters(models, oriSsTableDataStr, bagOfWordsModel)
        persistBloomFilters(bloomfilters, params)
    else:
        models = modelHelper.loadModel(namePrefix, params)
        bloomfilters = loadBloomFilters(params)
    print(params, "performing tests")
    (fprate,
    truePositive_c,truePositive_bf,truePositive,
    falsePositive_c,falsePositive_bf,falsePositive,
    trueNegative) = performTests(models, bloomfilters, oriSsTableDataStr, bagOfWordsModel)
    observations = (errorTrain, fprate,
                    truePositive_c,truePositive_bf,truePositive,
                    falsePositive_c,falsePositive_bf,falsePositive,
                    trueNegative)
    observationsdic = params.copy()
    observationsdic["errorTrain"] = errorTrain
    observationsdic["fpr"] = fprate
    observationsdic["truepositive_c"] = truePositive_c
    observationsdic["truepositive_bf"] = truePositive_bf
    observationsdic["truepositive"] = truePositive
    observationsdic["falsepositive_c"] = falsePositive_c
    observationsdic["falsepositive_bf"] = falsePositive_bf
    observationsdic["falsepositive"] = falsePositive
    observationsdic["truenegative"] = trueNegative
    persistObservations(observationsdic, params)
    endtime = time.time()
    return (endtime-starttime, params)



gramsize = -1
maxiter = 100
# models = {} # storing models based on sstable ids
# nu = 0.1
# gamma = 1
removeClassifier = False


# In[19]:


# thres = -0.1
# bloomfilters = {}
capacityBloom = 30000
errorRate = 0.3
skipFactor = 50


# In[20]:



In [9]:

# ssTableData = {}
separator = ":"
namePrefix = "svmsimple"
# namePrefix = sys.argv[3]
# namePrefixOri = "6nu6gamma_ex4"
namePrefixOri = namePrefix


#     (oriSsTableData, alltemptableData, ssTableData, oriSsTableDataStrSet, vectorizer, sstableBagData) = dataHelper.get()
oriSsTableDataStr, allSsTableDataStr, bagOfWordsModel = dataHelper.get()
# print(sstableBagData)

# scalers = getScalers()

# nus = np.logspace(-3,0,6)
# gammas = np.logspace(-3,3,6)
# nus = np.logspace(-3,0,6)
# gammas = np.logspace(-1,3,6)
num_cores = multiprocessing.cpu_count()

tempresults = ensemble(modelHelper.sampleparams, oriSsTableDataStr, bagOfWordsModel)
print(tempresults)
# results = Parallel(n_jobs=num_cores)(delayed(ensemble)(modelHelper.createParams(tuplethis)) for tuplethis in tqdm(modelHelper.getProductParams()))
# print(results)
# persistTimes(results)

1 405
2 880
3 397
4 845
5 368
{0: array(['ekVGu100:1747542952:000000000000FF80',
       'ekVGu100:1747542952:0000000000010000',
       'esadu100:1753044212:0000000000004100',
       'esadu100:1753044212:0000000000004200',
       'esadu100:1753044212:0000000000004280',
       'esadu100:1753044212:0000000000004380',
       'f0VRu100:1750161709:0000000000003B80',
       'f0VRu100:1750161709:0000000000004080',
       'f0VRu100:1750161709:0000000000004D00',
       'f0VRu100:1750161709:0000000000004D80',
       'fJzGu100:1747656599:0000000000005400',
       'fOzXt100:1736932149:0000000000000080',
       'fOzXt100:1736932149:0000000000000180',
       'fOzXt100:1736932149:0000000000000200',
       'fOzXt100:1736932149:0000000000000280',
       'fOzXt100:1736932149:0000000000000880',
       'fOzXt100:1736932149:0000000000000A00',
       'fOzXt100:1736932149:0000000000000A80',
       'fOzXt100:1736932149:0000000000000D80',
       'fOzXt100:1736932149:0000000000000E00',
       'fOzXt100:173693214

training done with sstableid -  1
0.1_100_rbf_0.001 time taken  0.0906517505645752
['1750162255:0000000000000D00' '1750162255:0000000000001400'
 '1750162255:0000000000001600' '1750162255:0000000000001980'
 '1750162255:0000000000001B00' '1750162255:0000000000001C00'
 '1750162255:0000000000002100' '1750162255:0000000000002180'
 '1750162255:0000000000002B00' '1750162255:0000000000003100'
 '1750162255:0000000000003280' '1750162255:0000000000003500'
 '1750162255:0000000000003800' '1750162255:0000000000004700'
 '1750162255:000000000000A100' '1750162255:000000000000A280'
 '1750162255:0000000000016600' '1752257917:0000000000001480'
 '1752257917:0000000000004600' '1752257917:0000000000004D80'
 '1752257917:0000000000005400' '1752257917:0000000000005680'
 '1752257917:0000000000005C00' '1752257917:0000000000005C80'
 '1752257917:0000000000005E00' '1752257917:0000000000006200'
 '1752257917:0000000000006300' '1752257917:0000000000008E00'
 '1752257917:0000000000008E80' '1752257917:0000000000008F80'
 '

--------------------------
newCapacity  528
bf creating for  2
['1750162255:0000000000000D00' '1750162255:0000000000001400'
 '1750162255:0000000000001600' '1750162255:0000000000001980'
 '1750162255:0000000000001B00' '1750162255:0000000000001C00'
 '1750162255:0000000000002100' '1750162255:0000000000002180'
 '1750162255:0000000000002B00' '1750162255:0000000000003100'
 '1750162255:0000000000003280' '1750162255:0000000000003500'
 '1750162255:0000000000003800' '1750162255:0000000000004700'
 '1750162255:000000000000A100' '1750162255:000000000000A280'
 '1750162255:0000000000016600' '1752257917:0000000000001480'
 '1752257917:0000000000004600' '1752257917:0000000000004D80'
 '1752257917:0000000000005400' '1752257917:0000000000005680'
 '1752257917:0000000000005C00' '1752257917:0000000000005C80'
 '1752257917:0000000000005E00' '1752257917:0000000000006200'
 '1752257917:0000000000006300' '1752257917:0000000000008E00'
 '1752257917:0000000000008E80' '1752257917:0000000000008F80'
 '1752257917:000000000

ValueError: invalid literal for int() with base 10: 'T'